In [1]:
!wget https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_India.zip

--2024-03-09 22:42:32--  https://bigdatacup.s3.ap-northeast-1.amazonaws.com/2022/CRDDC2022/RDD2022/Country_Specific_Data_CRDDC2022/RDD2022_India.zip
Resolving bigdatacup.s3.ap-northeast-1.amazonaws.com (bigdatacup.s3.ap-northeast-1.amazonaws.com)... 52.219.152.110, 3.5.157.185, 3.5.156.17, ...
Connecting to bigdatacup.s3.ap-northeast-1.amazonaws.com (bigdatacup.s3.ap-northeast-1.amazonaws.com)|52.219.152.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 526709145 (502M) [application/zip]
Saving to: 'RDD2022_India.zip'

RDD2022_India.zip   100%[===================>] 502.31M  41.8MB/s    in 14s     

2024-03-09 22:42:46 (36.9 MB/s) - 'RDD2022_India.zip' saved [526709145/526709145]



In [2]:
import bs4 as bs
import os
from pathlib import Path
import glob
from tqdm import tqdm
import random
import shutil

# Extracting Text from Xml files and Storing data for YOLO

In [78]:
ROOTDIR = "/kaggle/working/India/India/"
CountryListDir = "/kaggle/working/India/India/train/annotations/xmls"
fileList = sorted(glob.glob(CountryListDir+"/*.xml"))

In [79]:
fileList[5]

'/kaggle/working/India/India/train/annotations/xmls/India_000007.xml'

In [ ]:
import zipfile
with zipfile.ZipFile("/kaggle/working/RDD2022_India.zip", 'r') as zip_ref:
    zip_ref.extractall("/kaggle/working/India")

In [106]:
vocab = {
        "D00": 0,
        "D10": 1,
        "D20": 2,
        "D40": 3,
        "D01": 4,
        "D11": 5,
        "D43": 6,
        "D44": 7,
        "D50": 8
    }
name_list = list()
for n in range(len(fileList)):
    file = open(fileList[n], 'r')
    contents = file.read()
    if contents.find('object') != -1:
        objects = contents
        #name split
        name = objects.split('<name>')[1].split('</name>')[0]
        
        #width split
        image_width = int(objects.split('<width>')[1].split('</width>')[0])
        
        #height split
        image_height = int(objects.split('<height>')[1].split('</height>')[0])
        
        #xmin
        xmin = int(objects.split('<xmin>')[1].split('</xmin>')[0])
        
        #xmax
        xmax = int(objects.split('<xmax>')[1].split('</xmax>')[0])
        
        #ymin
        ymin = int(objects.split('<ymin>')[1].split('</ymin>')[0])        

        #ymax
        ymax = int(objects.split('<ymax>')[1].split('</ymax>')[0])
        
        w = (xmax - xmin)
        h = (ymax - ymin)
        cx = xmin + (w/2)
        cy = ymin + (h/2)

        # Normalize
        w = round((w / image_width), 4)
        h = round((h / image_height), 4)
        cx = round((cx / image_width), 4)
        cy = round((cy / image_height), 4)
        
        filename = objects.split('<filename>')[1].split('</filename>')[0].replace('.jpg','.txt')
        txtfile = open(outputDir + filename, 'w')
        txtfile.write(str(vocab.get(name,10)) + ' ' + str(w) + ' ' + str(h) +' ' + str(cx) + ' '+ str(cy))

In [6]:
def datasets(baseDir):
    random.seed(1337)
    baseOutputDir = "/kaggle/working/datasets/"
    countryName = Path(baseDir).parents[0]
    countryName = os.path.split(countryName)[1]

    baseImageDir = baseDir + "images/"
    baseAnnotDir = baseDir + "labels/"

    image_list_all = sorted(glob.glob(baseImageDir + "/*"))
    annot_list_all = sorted(glob.glob(baseAnnotDir + "/*"))

    backgroundImages_Percentage = 0.1
    image_list = []
    annot_list = []
    
    dataset_length_all = len(image_list_all)
    max_background_image = int(dataset_length_all*backgroundImages_Percentage)
    _counter = 0

    for i in range(len(annot_list_all)):
        
        with open(annot_list_all[i]) as f:
            _annot = f.read()

            # Annotation not empty
            if _annot:
                image_list.append(image_list_all[i])
                annot_list.append(annot_list_all[i])
            elif _counter < max_background_image:
                image_list.append(image_list_all[i])
                annot_list.append(annot_list_all[i])
                _counter = _counter + 1
                
    # Dataset length
    dataset_length = len(image_list)

    split_ratio = 0.9
    middle_point = round(split_ratio * dataset_length)

    numberList = list(range(0, dataset_length))
    random.shuffle(numberList)
    trainNumberList = numberList[:middle_point]
    validNumberList = numberList[middle_point:]
    print("Training/Validation Samples :", len(trainNumberList), len(validNumberList))

    print("Copying training images and labels for", countryName)
    for i in tqdm(trainNumberList):

        # Images
        outputImagesDir = baseOutputDir + countryName + "/images/train/"
        if not os.path.exists(outputImagesDir):
            os.makedirs(outputImagesDir)

        shutil.copy2(image_list[i], outputImagesDir)

        # Annotations
        outputAnnotDir = baseOutputDir + countryName + "/labels/train/"
        if not os.path.exists(outputAnnotDir):
            os.makedirs(outputAnnotDir)

        shutil.copy2(annot_list[i], outputAnnotDir)

    print("Copying validation images and labels for", countryName)
    for i in tqdm(validNumberList):

        # Images
        outputImagesDir = baseOutputDir + countryName + "/images/val/"
        if not os.path.exists(outputImagesDir):
            os.makedirs(outputImagesDir)

        shutil.copy2(image_list[i], outputImagesDir)

        # Annotations
        outputAnnotDir = baseOutputDir + countryName + "/labels/val/"
        if not os.path.exists(outputAnnotDir):
            os.makedirs(outputAnnotDir)

        shutil.copy2(annot_list[i], outputAnnotDir)

In [7]:
datasets("/kaggle/working/India/India/train/")

Training/Validation Samples : 4099 455
Copying training images and labels for India


100%|██████████| 4099/4099 [00:01<00:00, 3439.41it/s]


Copying validation images and labels for India


100%|██████████| 455/455 [00:00<00:00, 3423.80it/s]


# Training

In [9]:
! pip install ultralytics
! pip install opencv-python==4.8.0.74
! pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.1/720.1 kB 18.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 26.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.9.0.80
    Uninstalling opencv-python-4.9.0.80:
      Successfully uninstalled opencv-python-4.9.0.80
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.0 requires opencv-python>=4.9.0, but you have opencv-python 4.8.0.74 which is incompatible.


In [11]:
from ultralytics import YOLO

In [24]:
_project = "RDD_India"
_name = "RDD_YOLO_India_alone"
_data = "/kaggle/working/datasets/India1.yml"

# Epoch and warmup epoch
_epoch = 80
_warmup_epochs = int(_epoch*0.05)

model = YOLO('yolov8s.pt')
results = model.train(data=_data,
                      epochs=_epoch,
                      warmup_epochs=_warmup_epochs,
                      batch=32,
                      imgsz=640,
                      save_period=10,
                      workers=1,
                      project=_project,
                      name=_name,
                      seed=1337,
                      cos_lr=True,
                      mosaic=0.0,
                      
)

Ultralytics YOLOv8.1.25 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/datasets/India1.yml, epochs=80, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=1, project=RDD_India, name=RDD_YOLO_India_alone2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=1337, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

train: Scanning /kaggle/working/datasets/India/labels/train.cache... 4099 images, 681 backgrounds, 0 corrupt: 100%|██████████| 4099/4099 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/datasets/India/labels/val.cache... 455 images, 89 backgrounds, 0 corrupt: 100%|██████████| 455/455 [00:00<?, ?it/s]


Plotting labels to RDD_India/RDD_YOLO_India_alone2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 1 dataloader workers
Logging results to RDD_India/RDD_YOLO_India_alone2
Starting training for 80 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      46/80      14.5G      1.496      1.246      1.494          4        640: 100%|██████████| 129/129 [01:18<00:00,  1.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.13it/s]

                   all        455        794      0.554      0.355      0.425       0.19



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      47/80      14.4G      1.387      1.158      1.419         50        640:  10%|█         | 13/129 [00:07<01:06,  1.75it/s]


KeyboardInterrupt: 

## last.pt is our trained model

# IOU Score over some images

In [50]:
from torchvision.ops import box_iou
from PIL import Image 
import cv2
from typing import NamedTuple
import numpy as np

In [28]:
a = os.listdir("/kaggle/working/datasets/India/labels/train")

In [30]:
imgs = a[0:10]

In [51]:
CLASSES = [
    "Transverse Crack",
    "Alligator Crack",
    "Potholes",
    "Longitudinal Construction joint part",
    "Lateral Construction joint part",
    "Cross walk blur",
    "White line blur",
    "Utility Hole"
]


class Detection(NamedTuple):
    class_id: int
    label: str
    score: float
    box: np.ndarray

In [97]:
iou = 0.0
for i in imgs:
    image_file = Image.open("/kaggle/working/datasets/India/images/train/"+i.split(".txt")[0] + ".jpg")
    _image = np.array(image_file)
    h_ori = _image.shape[0]
    w_ori = _image.shape[1]
    
    resized_image = 0
    input_image = cv2.imread("/kaggle/working/datasets/India/images/train/"+i.split(".txt")[0] + ".jpg")
    if input_image is None:
        print("Error: Unable to load image")
    else:
        width = 640
        height = 640
    resized_image = cv2.resize(input_image, (width, height))
    results = model.predict(resized_image, conf=0.1)
    pred_box = torch.Tensor(results[0].boxes.xywhn)
    if(not pred_box.numel()):
        continue
    pred_boxes = pred_box[0]
    pred_boxes = pred_boxes.unsqueeze(0)
    with open("/kaggle/working/datasets/India/labels/train/"+i) as f:
        file = []
        for i in f:
            file.append([float(j) for j in i.split(" ")[1:]])
            break
    file_t = torch.Tensor(file).cuda()
    print(file_t, pred_boxes)
    #i_iou = box_iou(pred_boxes, file_t)
    #print(i_iou)


0: 640x640 1 Longitudinal Crack, 2 Alligator Cracks, 1 Potholes, 16.4ms
Speed: 2.5ms preprocess, 16.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
tensor([[0.7007, 0.8278, 0.3569, 0.1250]], device='cuda:0') tensor([[0.6985, 0.8383, 0.3583, 0.1451]], device='cuda:0')

0: 640x640 (no detections), 16.3ms
Speed: 1.7ms preprocess, 16.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 16.3ms
Speed: 1.7ms preprocess, 16.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Longitudinal Crack, 1 Alligator Crack, 1 Longitudinal Construction joint part, 16.3ms
Speed: 1.8ms preprocess, 16.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
tensor([[0.8403, 0.8250, 0.3194, 0.2333]], device='cuda:0') tensor([[0.8191, 0.8355, 0.3085, 0.2147]], device='cuda:0')

0: 640x640 4 Potholess, 16.3ms
Speed: 2.1ms preprocess, 16.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640

tensor([[377.0864, 467.2932, 638.9742, 575.7742],
        [137.8113, 474.6843, 377.0246, 571.9104],
        [145.8226, 473.2743, 521.1503, 574.0284],
        [195.6805,   0.0000, 327.1207, 244.0798]], device='cuda:0')